In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import os

In [105]:
file1="File_0_mine.h5"
file2="File_372.h5"
data1 = h5py.File(file1,'r')
data2 = h5py.File(file2,'r')

In [106]:
data1.keys()

<KeysViewHDF5 ['IC_charge', 'IC_charge_100ns', 'IC_charge_10ns', 'IC_charge_500ns', 'IC_charge_50ns', 'IC_charge_last', 'IC_diff', 'IC_first_charge', 'IC_mean', 'IC_mult', 'IC_num_pulses', 'IC_pulse_0_01_pct_charge_quantile', 'IC_pulse_0_03_pct_charge_quantile', 'IC_pulse_0_05_pct_charge_quantile', 'IC_pulse_0_08_pct_charge_quantile', 'IC_pulse_0_11_pct_charge_quantile', 'IC_pulse_0_15_pct_charge_quantile', 'IC_pulse_0_2_pct_charge_quantile', 'IC_pulse_0_5_pct_charge_quantile', 'IC_pulse_0_8_pct_charge_quantile', 'IC_time_first', 'IC_time_last', 'IC_time_spread', 'IC_time_std', 'IC_time_weighted_median', 'IC_var', 'reco_vals']>

In [107]:
data2.keys()

<KeysViewHDF5 ['IC_charge', 'IC_charge_100ns', 'IC_charge_10ns', 'IC_charge_500ns', 'IC_charge_50ns', 'IC_charge_last', 'IC_diff', 'IC_first_charge', 'IC_mean', 'IC_mult', 'IC_num_pulses', 'IC_pulse_0_01_pct_charge_quantile', 'IC_pulse_0_03_pct_charge_quantile', 'IC_pulse_0_05_pct_charge_quantile', 'IC_pulse_0_08_pct_charge_quantile', 'IC_pulse_0_11_pct_charge_quantile', 'IC_pulse_0_15_pct_charge_quantile', 'IC_pulse_0_2_pct_charge_quantile', 'IC_pulse_0_5_pct_charge_quantile', 'IC_pulse_0_8_pct_charge_quantile', 'IC_time_first', 'IC_time_last', 'IC_time_spread', 'IC_time_std', 'IC_time_weighted_median', 'IC_var', 'reco_vals']>

file1 contains 'IC_diff', 'IC_mean', 'IC_mult' file2 not

In [108]:
data1['IC_mean']

<HDF5 dataset "IC_mean": shape (86, 10, 10, 60, 1), type "<f8">

In [109]:
data2['reco_vals']

<HDF5 dataset "reco_vals": shape (146,), type "|V520">

In [110]:
data1['IC_first_charge']

<HDF5 dataset "IC_first_charge": shape (86, 10, 10, 60, 1), type "<f8">

In [8]:
data2['IC_first_charge']

<HDF5 dataset "IC_first_charge": shape (135277, 10, 10, 60, 1), type "<f8">

In [9]:
135277/40

3381.925

In [3]:
file3="File_372.h5"
data3 = h5py.File(file3,'r')
data3.keys()

<KeysViewHDF5 ['IC_charge', 'IC_charge_100ns', 'IC_charge_10ns', 'IC_charge_500ns', 'IC_charge_50ns', 'IC_charge_last', 'IC_diff', 'IC_first_charge', 'IC_mean', 'IC_mult', 'IC_num_pulses', 'IC_pulse_0_01_pct_charge_quantile', 'IC_pulse_0_03_pct_charge_quantile', 'IC_pulse_0_05_pct_charge_quantile', 'IC_pulse_0_08_pct_charge_quantile', 'IC_pulse_0_11_pct_charge_quantile', 'IC_pulse_0_15_pct_charge_quantile', 'IC_pulse_0_2_pct_charge_quantile', 'IC_pulse_0_5_pct_charge_quantile', 'IC_pulse_0_8_pct_charge_quantile', 'IC_time_first', 'IC_time_last', 'IC_time_spread', 'IC_time_std', 'IC_time_weighted_median', 'IC_var', 'reco_vals']>

In [4]:
data3['IC_first_charge']

<HDF5 dataset "IC_first_charge": shape (146, 10, 10, 60, 1), type "<f8">

In [2]:
import tables
import os

In [3]:
input_shape = [10, 10, 60]
FILTERS = tables.Filters(complib='zlib', complevel=9)
   #os.listdir(".")[5]
tfile=['File_372.h5','File_0_mine.h5']
print('Try to open {}'.format(tfile))
hf1 = h5py.File(tfile[0], 'r')

Try to open ['File_372.h5', 'File_0_mine.h5']


In [ ]:
d = "dir"
for path in os.listdir(d):
    full_path = os.path.join(d, path)
    if os.path.isfile(full_path):
        print full_path

In [7]:
"d"+["f","a"]

TypeError: can only concatenate str (not "list") to str

In [100]:
keys = list(hf1.keys())
dtype=hf1["reco_vals"].dtype
args = dict()
args['outfile'] = 'outmerged2.h5'


with tables.open_file(args['outfile'], mode="w", title="Events for training the NN", filters=FILTERS) as h5file:
    
    input_features = []
    for okey in keys[:-1]:
        feature = h5file.create_earray(
                h5file.root, okey, tables.Float64Atom(),
                (0, input_shape[0], input_shape[1], input_shape[2], 1),
                title=okey)
        feature.flush()
        input_features.append(feature)
    reco_vals = tables.Table(h5file.root, 'reco_vals', description=dtype)
    h5file.root._v_attrs.shape = input_shape
    hf1.close()
    for fili in tfile:
        print('Open {}'.format(fili))
        one_h5file = h5py.File(fili, 'r')
        num_events = len(one_h5file["reco_vals"])
        for k in range(num_events):
            for i, okey in enumerate(keys[:-1]):
                input_features[i].append(np.expand_dims(one_h5file[okey][k], axis=0))
            reco_vals.append(np.atleast_1d(one_h5file["reco_vals"][k]))
        for inp_feature in input_features:
                inp_feature.flush()
        reco_vals.flush()
    one_h5file.close()


Open File_372.h5
Open File_0_mine.h5


In [17]:
file2="../h5_merged4.h5"
data1 = h5py.File(file2,'r')

In [18]:
data1.keys()

<KeysViewHDF5 ['IC_charge', 'IC_charge_100ns', 'IC_charge_10ns', 'IC_charge_500ns', 'IC_charge_50ns', 'IC_charge_last', 'IC_diff', 'IC_first_charge', 'IC_mean', 'IC_mult', 'IC_num_pulses', 'IC_pulse_0_01_pct_charge_quantile', 'IC_pulse_0_03_pct_charge_quantile', 'IC_pulse_0_05_pct_charge_quantile', 'IC_pulse_0_08_pct_charge_quantile', 'IC_pulse_0_11_pct_charge_quantile', 'IC_pulse_0_15_pct_charge_quantile', 'IC_pulse_0_2_pct_charge_quantile', 'IC_pulse_0_5_pct_charge_quantile', 'IC_pulse_0_8_pct_charge_quantile', 'IC_time_first', 'IC_time_last', 'IC_time_spread', 'IC_time_std', 'IC_time_weighted_median', 'IC_var', 'reco_vals']>

In [21]:
data1['IC_mean']

<HDF5 dataset "IC_mean": shape (37205, 10, 10, 60, 1), type "<f8">